# found in 
https://github.com/philtabor/Youtube-Code-Repository/blob/master/ReinforcementLearning/DeepQLearning/ddqn_keras.py


import os
# for keras the CUDA commands must come before importing the keras libraries
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import gym

env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

ddqn_agent = DDQNAgent(alpha=0.0005, gamma=0.99, n_actions=action_size, epsilon=1.0,
              batch_size=64, input_dims=state_size)
n_games = 500
ddqn_scores = []
eps_history = []


for i in range(n_games):
    done = False
    score = 0
    observation = env.reset()
    for time in range(500):
        action = ddqn_agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        reward = reward if not done else -10
        score += reward
        ddqn_agent.remember(observation, action, reward, observation_, int(done))
        observation = observation_
        ddqn_agent.learn()
        if done:
            break
            
    eps_history.append(ddqn_agent.epsilon)

    ddqn_scores.append(score)

    avg_score = np.mean(ddqn_scores[max(0, i-100):(i+1)])
    print('episode: ', i,'score: %.2f' % score,
          ' average score %.2f' % avg_score)

    if i % 10 == 0 and i > 0:
        ddqn_agent.save_model()


x = [i+1 for i in range(n_games)]


In [1]:
# handling colab problem
import os
if not 'data' in os.listdir('..'):
    print('needs data folder. imports through git')
    !git clone https://github.com/JakartaLaw/speciale.git
    print(os.listdir())
    os.chdir('speciale//src')


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from environments import EnvironmentModel1
from environments import translate_action_model1 as translate_action
from environments import reward_scaler_model1 as reward_scaler

from agents import DDQNAgent

In [4]:
def draw_beta():
    beta_L, beta_K = np.random.uniform(0.2, 4), np.random.uniform(0.2, 4)
    return {'beta_L' : beta_L, 'beta_K': beta_K}


In [5]:
parameters = {
    'beta_K' : 1,
    'beta_L' : 1,
    'sigma_epsilon' : 0.1, 
    'S_min': 120.0,
    'alpha': 4.609,
    'eta_G': 0.164,
    'eta_G_sq' : 0.015,
    'delta': 0.209,
    'sigma_epsilon': 15.11
}
env = EnvironmentModel1(**parameters)

In [7]:
state_size = env.observation_space.shape[0]
action_size = 4
ddqn_agent = DDQNAgent(alpha = 0.0005, gamma = 0.99, epsilon=1.0, input_dims=state_size, n_actions=action_size,
                  batch_size=64)
done = False
EPISODES = 1000



ddqn_scores = []
eps_history = []


for i in range(EPISODES):
    done = False
    score = 0
    observation = env.reset()
    
    while not done:
        #print(observation)
        action = ddqn_agent.choose_action(observation)
        observation_, reward, done, info = env.step(action)
        score += reward
        scaled_reward = reward_scaler(reward, env.beta_K, env.beta_L)
        ddqn_agent.remember(observation, action, scaled_reward, observation_, int(done))
        observation = observation_
        ddqn_agent.learn()

            
    eps_history.append(ddqn_agent.epsilon)

    ddqn_scores.append(score)

    avg_score = np.mean(ddqn_scores[max(0, i-100):(i+1)])
    print('episode: ', i,'score: %.2f' % score,
          ' average score %.2f' % avg_score)

    if i % 10 == 0 and i > 0:
        pass
        #ddqn_agent.save_model()

episode:  0 score: 984.87  average score 984.87
episode:  1 score: 941.12  average score 962.99
episode:  2 score: 971.15  average score 965.71
episode:  3 score: 958.92  average score 964.01
episode:  4 score: 956.59  average score 962.53


KeyboardInterrupt: 